In [1]:
import pandas as pd
from datetime import datetime

In [2]:
course_skills_df = pd.read_csv(r'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/raw/course_skills.csv')
user_skills_df = pd.read_csv(r'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/raw/user_skills.csv')
courses_df = pd.read_csv(r'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/transformed/transformed_courses.csv')
progress_df = pd.read_csv(r'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/transformed/transformed_progress.csv')
users_df = pd.read_csv(r'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/transformed/transformed_users.csv')

In [3]:
fact_courses_file = 'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/.venv/mart/dimension/fact_courses.csv'

In [4]:
progress_df['updated_date'] = pd.to_datetime(
    progress_df['updated_year'].astype(str) + '-' +
    progress_df['updated_month'].astype(str) + '-' +
    progress_df['updated_day'].astype(str)
)


In [5]:
# Calculate the completion time for each user in each course
completion_times = []

for course_id, group in progress_df.groupby('courseId'):
    # Get the start date (when the user first starts the course, percentage 0)
    start_dates = group[group['percentage_completed'] == 0][['userId', 'updated_date']]
    
    # Get the end date (when the user completes the course, percentage 100)
    end_dates = group[group['percentage_completed'] == 100][['userId', 'updated_date']]
    
    # Merge start and end dates on userId
    merged_dates = start_dates.merge(end_dates, on='userId', suffixes=('_start', '_end'))
    
    # Calculate the time taken for each user to complete the course
    merged_dates['days_to_complete'] = (merged_dates['updated_date_end'] - merged_dates['updated_date_start']).dt.days
    
    # Append course_id and the days to complete for each user
    for _, row in merged_dates.iterrows():
        completion_times.append((course_id, row['userId'], row['days_to_complete']))

# Create a DataFrame from completion times
completion_df = pd.DataFrame(completion_times, columns=['courseId', 'userId', 'days_to_complete'])

# Calculate the total days and the number of unique users who completed the course
course_completion_stats = completion_df.groupby('courseId').agg(
    total_days_to_complete=('days_to_complete', 'sum'),    # Total days taken to complete
    total_users_completed=('userId', 'nunique')            # Count of unique users who completed the course
).reset_index()

# Calculate average days to complete
course_completion_stats['average_days_to_complete'] = course_completion_stats['total_days_to_complete'] / course_completion_stats['total_users_completed']

# Merge with courses DataFrame
fact_courses_df = courses_df.merge(course_completion_stats[['courseId', 'average_days_to_complete', 'total_users_completed']], on='courseId', how='left')

# Fill NaN values in average_days_to_complete with 0 for courses not completed by anyone
fact_courses_df['average_days_to_complete'].fillna(0, inplace=True)

# Display the resulting fact_courses DataFrame to check available columns
print("Fact Courses DataFrame:\n", fact_courses_df.head())
print("Columns in fact_courses_df:\n", fact_courses_df.columns)

Fact Courses DataFrame:
    courseId                                          title proficiency_level  \
0         1                    Java Programming Essentials          Beginner   
1         2  Mastering Java Swing for Desktop Applications          Beginner   
2         3                        SQL for Data Management          Advanced   
3         4                    C Programming for Beginners          Beginner   
4         5                         Advanced C# Techniques          Advanced   

   chapters  duration  course_age duration_category  \
0         5         2        1634             Short   
1         9        10        1487              Long   
2        10        10         150              Long   
3        15         7        1096            Medium   
4         7         5        1321            Medium   

   proficiency_level_encoded  created_day  created_month  created_year  \
0                          1           18              4          2020   
1              

C:\Users\AnanyaSarkar\AppData\Local\Temp\ipykernel_21568\1540376115.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fact_courses_df['average_days_to_complete'].fillna(0, inplace=True)


In [6]:
print(fact_courses_df[['average_days_to_complete', 'total_users_completed']].head())


   average_days_to_complete  total_users_completed
0                610.449275                    138
1                606.933333                     75
2                 23.274725                     91
3                273.213115                     61
4                451.351852                    108


In [7]:
# Save the final dimension table to CSV
fact_users_df.to_csv('C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/marts/fact/fact_users.csv', index=False)

print("fact_users.csv created successfully created ")

NameError: name 'fact_users_df' is not defined